![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [1]:
# Start your code here!
import pandas as pd

In [2]:
ds_jobs = pd.read_csv('customer_train.csv')
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [3]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [4]:
ds_jobs_clean = ds_jobs.copy()

In [5]:
ordered_cats ={
    'relevant_experience': ['No relevant experience', 'Has relevant experience'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

In [6]:
for col in ds_jobs_clean:
    # convert int to int 32
    if ds_jobs_clean[col].dtype=='int':
        ds_jobs_clean[col]=ds_jobs_clean[col].astype('int32')
    # convert float to float16    
    elif ds_jobs_clean[col].dtype=='float':
        ds_jobs_clean[col]=ds_jobs_clean[col].astype('float16')
    # convert ordered categorical to ordered categories
    elif col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        ds_jobs_clean[col] = ds_jobs_clean[col].astype(category)
    # convert other to standard categories
    else:
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('category')

In [7]:
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]

In [8]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   category
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   category
 13  job_change              2201 non-null   category
dtypes: category(13), float1

In [9]:
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0


In [10]:
ds_jobs_clean.isna().sum()

student_id                  0
city                        0
city_development_index      0
gender                    380
relevant_experience         0
enrolled_university        16
education_level            17
major_discipline          104
experience                  0
company_size                0
company_type               57
last_new_job               17
training_hours              0
job_change                  0
dtype: int64

In [11]:
ds_jobs_clean['gender'].value_counts()

Male      1654
Female     145
Other       22
Name: gender, dtype: int64

In [12]:
missing_features = ['gender', 'enrolled_university', 'education_level', 'major_discipline', 'company_type', 'last_new_job']

for col in missing_features:
    most_freq = ds_jobs_clean[col].mode()[0]
    ds_jobs_clean[col].fillna(most_freq, inplace=True)


In [13]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   category
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  2201 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2201 non-null   category
 6   education_level         2201 non-null   category
 7   major_discipline        2201 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2201 non-null   category
 11  last_new_job            2201 non-null   category
 12  training_hours          2201 non-null   category
 13  job_change              2201 non-null   category
dtypes: category(13), float1

In [14]:
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0


In [15]:
new_df = ds_jobs_clean.copy()

for col in new_df.select_dtypes(include='category').columns:
    new_df[col] = new_df[col].cat.codes

print(new_df.head())

    student_id  city  city_development_index  gender  relevant_experience  \
9          375     5                0.919922       1                    1   
12       14634    91                0.913086       1                    1   
31       12652     5                0.919922       1                    1   
34       15126    48                0.910156       1                    1   
40        1388    13                0.925781       0                    1   

    enrolled_university  education_level  major_discipline  experience  \
9                     0                2                 5          17   
12                    0                2                 5          21   
31                    1                2                 5          19   
34                    0                2                 1          12   
40                    2                3                 5          16   

    company_size  company_type  last_new_job  training_hours  job_change  
9              7 

In [16]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              2201 non-null   int16  
 1   city                    2201 non-null   int8   
 2   city_development_index  2201 non-null   float16
 3   gender                  2201 non-null   int8   
 4   relevant_experience     2201 non-null   int8   
 5   enrolled_university     2201 non-null   int8   
 6   education_level         2201 non-null   int8   
 7   major_discipline        2201 non-null   int8   
 8   experience              2201 non-null   int8   
 9   company_size            2201 non-null   int8   
 10  company_type            2201 non-null   int8   
 11  last_new_job            2201 non-null   int8   
 12  training_hours          2201 non-null   int16  
 13  job_change              2201 non-null   int8   
dtypes: float16(1), int16(2), int8(11)
memor

In [17]:
X = new_df.drop(['job_change', 'student_id'], axis=1)
y = new_df['job_change']

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=99)

In [19]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=9)

X_train_resample, y_train_resample = smote.fit_resample(X_train, y_train)
X_test_resample, y_test_resample = smote.fit_resample(X_test, y_test)

In [20]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train_resample, y_train_resample)

c:\Users\Aryo Sasi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
y_pred = lr.predict(X_test_resample)

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test_resample, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.66      0.68       382
           1       0.68      0.71      0.69       382

    accuracy                           0.69       764
   macro avg       0.69      0.69      0.69       764
weighted avg       0.69      0.69      0.69       764



In [23]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

model_params = {
    'svc': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20]
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [24]:
from sklearn.model_selection import GridSearchCV

scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv = 5, return_train_score=False)
    clf.fit(X_train_resample, y_train_resample)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })

In [25]:
df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svc,0.884008,{'C': 20}
1,random_forest,0.866667,{'n_estimators': 10}
2,logistic_regression,0.702180,{'C': 5}


In [26]:
svc = SVC(C=20)
svc.fit(X_train, y_train)

SVC(C=20)

In [27]:
y_pred = svc.predict(X_test)

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93       382
           1       0.00      0.00      0.00        59

    accuracy                           0.87       441
   macro avg       0.43      0.50      0.46       441
weighted avg       0.75      0.87      0.80       441



c:\Users\Aryo Sasi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Aryo Sasi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Aryo Sasi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
